In [6]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [7]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [13]:
df = pd.read_csv('17SDGpt2_tweets.csv', lineterminator='\n', index_col = 0)

In [14]:
def decontracted(phrase):
    """decontracts the words to its expanded form"""
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [15]:
import re

def clean_text(df):
    df = df.copy().reset_index(drop = True)
     
    #decontract
    df = df.apply(decontracted)
    
    df = df.apply(lambda x: re.sub(r"http\S+", "", x), 1)\
.apply(lambda i: " ".join(filter(lambda x:x[0]!="@", i.split())), 1)\
.apply(lambda x: re.sub(r"&amp", "",x),1)\
.apply(lambda x: re.sub(r"&amp;", "",x),1)\
.apply(lambda x: str(x).lower()).replace('\\', '').replace('_', ' ')\
.apply(lambda x: re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', x),1)

    return df

In [ ]:
def clean_sentence(string):
    string = re.sub()

In [16]:
df['clean_tweet'] = clean_text(df['tweet'])
df['clean_tweet'] = df['clean_tweet'].drop_duplicates()
df = df[~df['clean_tweet'].isnull()]
df = df[['sdg', 'clean_tweet']]

In [17]:
df

,sdg,clean_tweet
0,16,our spousal sponsorship application sent to th...
1,16,star wars the vintage collection vc186 boba fe...
2,16,cds is hiring a team lead for our delivery pol...
3,16,was watching him on show… a person that was re...
4,16,as written xiv directly contradicts medical a...
...,...,...
162355,17,you dont care about all the people who are los...
162356,17,to be a guest for what how to smoke crack or...
162357,17,mba alumni association announces an official c...
162358,17,they are feeling it as far away as australia ...


In [18]:
def get_vec(x):
    doc = nlp(x)
    vec = doc.vector
    return vec

In [19]:
df['vec'] = df['clean_tweet'].apply(lambda x: get_vec(x))

In [20]:
df['vec']

0         [-0.13250092, -0.016407665, 0.030081144, -0.07...
1         [0.044625, -0.064564094, 0.12102854, -0.076023...
2         [-0.104450434, 0.17593025, 0.001941288, -0.024...
3         [-0.011462458, 0.08298158, -0.1042032, -0.0250...
4         [-0.04601867, 0.053797245, -0.13446185, 0.0557...
                                ...                        
162355    [-0.09476761, 0.08401415, -0.17036651, -0.1726...
162356    [-0.069012105, 0.105398305, -0.15936907, 0.064...
162357    [0.008951428, 0.052970894, 0.042005144, 0.0033...
162358    [-0.010249723, 0.13045676, -0.10021108, -0.068...
162359    [-0.12701419, 0.1285858, -0.18099453, 0.019388...
Name: vec, Length: 148640, dtype: object

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148640 entries, 0 to 162359
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sdg          148640 non-null  int64 
 1   clean_tweet  148640 non-null  object
 2   vec          148640 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.5+ MB


In [24]:
df.to_json('vec_df.json')

In [54]:
X = df['vec'].to_numpy()
print(X.shape)
X = X.reshape(-1, 1)
print(X.shape)

(148640,)
(148640, 1)


In [55]:
y = df['sdg']
y.shape

(148640,)

In [56]:
X_vec = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [57]:
X_vec.shape

(148640, 300)

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size = 0.4, random_state = 777, stratify = y)

In [59]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((89184, 300), (59456, 300), (89184,), (59456,))

In [60]:
clf = LogisticRegression(solver='liblinear')

In [61]:
clf.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [62]:
y_pred = clf.predict(X_test)

In [63]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.73      0.75      0.74      2139
           2       0.63      0.57      0.60      2242
           3       0.43      0.40      0.41      4194
           4       0.67      0.85      0.75      8339
           5       0.65      0.73      0.68      3004
           6       0.62      0.54      0.58      2130
           7       0.71      0.68      0.69      3004
           8       0.50      0.43      0.46      3139
           9       0.50      0.60      0.55      4840
          10       0.55      0.42      0.48      2987
          11       0.48      0.40      0.44      3388
          12       0.67      0.68      0.67      3886
          13       0.61      0.54      0.57      2624
          14       0.60      0.43      0.50      2030
          15       0.58      0.63      0.60      4806
          16       0.45      0.44      0.44      4024
          17       0.55      0.46      0.50      2680

    accuracy              

In [64]:
import pickle

pickle.dump(clf, open('w2v_sdg.pkl', 'wb'))
model = pickle.load(open('w2v_sdg.pkl', 'rb'))


In [65]:
sentence = "the worlds oceans their temperature chemistry currents and life drive global systems that make the earth habitable for humankind"

vec = get_vec(sentence)
vec

array([-3.26057933e-02,  1.09173112e-01, -4.95378971e-02, -2.50302106e-02,
        4.39413190e-02,  5.57704680e-02, -1.01618037e-01,  2.91453928e-01,
       -3.65166329e-02,  1.89926887e+00, -3.44099961e-02, -1.47595420e-01,
        6.94541559e-02,  6.85945898e-02, -1.23440921e-01, -3.41126323e-02,
       -4.10271734e-02,  1.40386105e+00, -2.29337186e-01, -5.78262694e-02,
        1.98166054e-02, -5.29878139e-02, -5.37974350e-02,  8.42481554e-02,
        1.19209640e-01,  6.53674724e-05,  7.48045743e-02,  2.61125285e-02,
       -1.71874575e-02,  1.87466905e-01, -1.42906711e-01,  7.95199573e-02,
       -1.23316366e-02, -7.43202567e-02,  2.04405338e-01, -1.19356930e-01,
        8.59363005e-03, -8.17679539e-02,  7.96529725e-02,  7.76848421e-02,
        3.01454850e-02, -7.42799137e-03, -3.35640572e-02,  2.71183904e-02,
       -5.54900523e-03, -1.54427320e-01, -1.75522953e-01, -6.12386949e-02,
       -1.21798165e-01, -2.84503195e-02,  1.45767048e-01,  2.73254830e-02,
       -5.82536273e-02, -

In [66]:
model.predict(vec.reshape(1, -1))

array([14], dtype=int64)

In [67]:
df2 = pd.read_csv('val_set_sdg_1_7_8_12_13_toy.csv')

In [70]:
df2 = df2[['abstract','sdg_id']]

'this study examines the challenges that recent college graduates face in a hard-hit us region during the great recession'

In [83]:
sentence2 = df2['abstract'][0].lower()
sen2 = sentence2.split(".")[0]
vec2 = get_vec(sen2)
print(sen2)
vec2

this study examines the challenges that recent college graduates face in a hard-hit us region during the great recession


array([-3.32921110e-02,  1.64053425e-01,  4.52050902e-02,  5.10878973e-02,
        1.39066964e-01, -1.30030423e-01, -3.96095887e-02,  1.52444139e-01,
        9.68620554e-02,  2.54028630e+00, -2.83333570e-01, -7.06118643e-02,
        9.73857269e-02, -7.58929476e-02,  1.87340975e-02,  5.06655499e-03,
        1.46654276e-02,  1.00508606e+00, -1.52974054e-01, -8.74854304e-05,
        4.76050898e-02, -1.51956424e-01,  3.06519493e-02, -4.84079532e-02,
        6.21650405e-02,  7.59190917e-02,  1.92214479e-03,  1.51302606e-01,
       -1.80323631e-01,  5.13542145e-02, -9.44124982e-02,  1.03282824e-01,
       -8.22580904e-02, -1.55418053e-01,  2.25093663e-01,  3.97330970e-02,
        3.15382592e-02, -9.68629587e-03, -3.54588889e-02, -1.66158393e-01,
       -7.24428450e-04,  8.45139101e-02, -7.91781116e-03, -8.34692866e-02,
       -7.95729980e-02,  8.16960409e-02, -7.05227628e-02,  4.35886383e-02,
        1.42216170e-02, -6.32526502e-02, -1.10304654e-01,  1.25802577e-01,
       -2.16859486e-02, -

In [84]:
model.predict(vec2.reshape(1, -1))

array([4], dtype=int64)